# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    x = (data["Price in 1 Year"][0] / P) - 1
    y = (data["Price in 1 Year"][1] / P) - 1
    z = (data["Price in 1 Year"][2] / P) - 1
    Px = data["Probability"][0]
    Py = data["Probability"][1]
    Pz = data["Probability"][2]
    return (x * Px) + (y * Py) + (z * Pz)

0.07500000000000002


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
import math

def volatility(data, P):
    x = (data["Price in 1 Year"][0] / P) - 1
    y = (data["Price in 1 Year"][1] / P) - 1
    z = (data["Price in 1 Year"][2] / P) - 1
    Px = data["Probability"][0]
    Py = data["Probability"][1]
    Pz = data["Probability"][2]
    exp = exp_return(data, 40)
    var = (((x - exp) **2) * Px) + (((y - exp) **2) * Py) + (((z - exp) **2) * Pz)
    return math.sqrt(var)

0.21301408404140795


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    st_error = (volatility(data, P))/np.sqrt(len(data))
    upper_bound = exp_return(data, P) + (st_error * 1.96)
    lower_bound = exp_return(data, P) - (st_error * 1.96)
    return [lower_bound, upper_bound]

[-0.16604812797447732, 0.3160481279744774]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    expected_return = rf + beta * (market_return - rf)
    return expected_return

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    import pandas as pd
    df = pd.read_csv(path)

    df = df[pd.to_numeric(df['RET'], errors='coerce').notna()]

    security_counts = df.groupby('PERMNO')['RET'].count().reset_index()

    security_counts = security_counts[security_counts['RET'] >= 500]

    df = df[df['PERMNO'].isin(security_counts['PERMNO'])]

    df = df[['PERMNO', 'date', 'CUSIP', 'RET']]

    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    clean = clean_data(data)
    avg_return_data = clean.groupby('CUSIP')
    a = []
    b = []
    for x, y in avg_return_data:
        d = []
        for n in y['RET']:
            d+= [float(n)]
        num = sum(d)
        denom = len(d)
        a += [num/denom]
        b += [x]
    
    output = pd.DataFrame(data = {
        "CUSIP" : b,
        "Average Return" : a
    })
    
    return output

CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]

In [12]:
def security_volatilities(data):
    clean = clean_data(data)
    security_volatilities_data = clean.groupby('CUSIP')
    a = []
    b = []
    i = 0
    s = []
    for x, y in security_volatilities_data:
        d = []
        for n in y['RET']:
            d+= [float(n)]
        num = sum(d)
        denom = len(d)
        a += [num/denom]
        b += [x]
    for l, m in security_volatilities_data:
        numerator = 0
        for e in m['RET']:
            numerator += (float(e)-a[i])**2
        i+= 1
        denominator = len(m['RET'])
        s += [np.sqrt(numerator/(denominator-1))]
        
        

    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : b,
        "Volatility" : s
    })

    return output


CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    u = []
    q = []
    for r in ret_data["Average Return"]:
        u += [r]
    for t in vol_data["Volatility"]:
        q += [t]
    w = sum(q) / len(q)
    x = sum(u) / len(u)
    y = []
    v = []
    for f in u:
        y+=[f-x]
    for z in q:
        v+=[z-w]
    numerator = []
    for g in range(0, len(y)):
        numerator+= [y[g]*v[g]]
    num = sum(numerator)
    a = 0
    for i in y:
        a+=i**2
    b = 0
    for j in v:
        b+=j**2
    corr = num / np.sqrt(a*b)
        
    
    return corr

0.44837090727059614
